# Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load Data

In [ ]:
# Load and preprocess data
file_path = r'C:\Users\File.csv'
data = pd.read_csv(file_path)
data = data.dropna(subset=['Org'])

# Data Mapping

In [ ]:
# Map each unique 'Source' IP to a Server ID
unique_servers = data['Source'].unique()
server_mapping = {ip: idx for idx, ip in enumerate(unique_servers)}
data['ServerID'] = data['Source'].map(server_mapping)
print("\nServer Mapping (Source IP to Server ID):")
print(server_mapping)

# Sort & Sequence

In [ ]:
# Convert AdjustedTime to numeric format
data['AdjustedTime'] = pd.to_datetime(data['AdjustedTime']).astype('int64') // 10**9  # Convert to seconds

# Encode categorical columns
label_encoders = {}
categorical_cols = ['Protocol', 'Connection', 'User', 'Org']
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

# Define feature and target columns
features = ['AdjustedTime', 'Protocol', 'Connection', 'User', 'Length', 'ARTT', 'Longitude', 'Latitude', 'Org']
labels = 'ServerID'

# Scale features
scaler = StandardScaler()
data[features] = scaler.fit_transform(data[features])

# Group by user to create sequences
user_groups = data.groupby('User')
X_sequences = []
y_sequences = []
sequence_length = 60

for user, group in user_groups:
    user_features = group[features].values
    user_labels = group[labels].values
    
    # Create rolling sequences
    for i in range(len(user_features) - sequence_length + 1):
        X_sequences.append(user_features[i:i + sequence_length])
        y_sequences.append(user_labels[i + sequence_length - 1])  # Target is last ID in the sequence

# Convert lists to numpy arrays
X_sequences = pad_sequences(X_sequences, maxlen=sequence_length, dtype='float32')
y_sequences = np.array(y_sequences)


# Train-Test Split

In [ ]:
# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.15, random_state=42)

# Transformer Model Defintion

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, Lambda
from tensorflow.keras.models import Model

# Positional Encoding Function
def positional_encoding(sequence_length, feature_dim):
    positions = np.arange(sequence_length)[:, np.newaxis]  # Shape: (sequence_length, 1)
    dimensions = np.arange(feature_dim)[np.newaxis, :]  # Shape: (1, feature_dim)
    
    angle_rates = 1 / np.power(10000, (2 * (dimensions // 2)) / np.float32(feature_dim))
    angle_rads = positions * angle_rates

    # Apply sin to even indices and cos to odd indices
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    return tf.cast(angle_rads, dtype=tf.float32)

# Transformer Encoder Block
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)
        ])

    def call(self, inputs, training=False):
        attn_output = self.attention(inputs, inputs)  # Self-attention
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.norm1(inputs + attn_output)  # Skip connection + Norm
        
        ffn_output = self.ffn(out1)  # Feed-forward
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.norm2(out1 + ffn_output)  # Final skip connection + Norm

# Model Architecture
sequence_length = X_train.shape[1]  # 60
feature_dim = X_train.shape[2]  # 9 (assuming 9 features per time step)
num_classes = len(unique_servers)

# Encoder Input
encoder_inputs = Input(shape=(sequence_length, feature_dim))

# Apply Positional Encoding as a Layer
pos_encoding = positional_encoding(sequence_length, feature_dim)
pos_encoding_layer = Lambda(lambda x: x + pos_encoding)(encoder_inputs)

# Stack Transformer Encoder Blocks
encoder_output = TransformerEncoder(embed_dim=feature_dim, num_heads=4, ff_dim=128)(pos_encoding_layer)
encoder_output = TransformerEncoder(embed_dim=feature_dim, num_heads=4, ff_dim=128)(encoder_output)

# Global Average Pooling to reduce sequence dimension
encoder_output = GlobalAveragePooling1D()(encoder_output)

# Fully connected layers
x = Dense(64, activation='relu')(encoder_output)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)

# Define the model (Single input now)
model = Model(encoder_inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


# Training

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.176, batch_size=32, verbose=1)

# Evaluate

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Model accuracy on test set:", accuracy)